In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

train_data = pd.read_json('../data/processed/train.json')

In [2]:
train_data_band_1 = np.array([np.array(x).reshape(75, 75) for x in train_data['band_1']])
train_data_band_2 = np.array([np.array(x).reshape(75, 75) for x in train_data['band_2']])
train_data_X = np.concatenate([train_data_band_1[:, :, :, np.newaxis], train_data_band_2[:, :, :, np.newaxis]], axis=-1)
train_data_y = train_data['is_iceberg'].values

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data_X, train_data_y, train_size=0.8)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


# First Attempt : CNN

In [11]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(75, 75, 2), activation='relu', data_format='channels_last'))
model.add(Dropout(0.2))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 73, 73, 32)        608       
_________________________________________________________________
dropout_19 (Dropout)         (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 71, 71, 32)        9248      
_________________________________________________________________
dropout_20 (Dropout)         (None, 71, 71, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 161312)            0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               41296128  
_________________________________________________________________
dropout_21 (Dropout)         (None, 256)               0         
__________

In [12]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Train on 1283 samples, validate on 321 samples
Epoch 1/10


KeyboardInterrupt: 